<a href="https://colab.research.google.com/github/vincm1/RecSys_Implicit/blob/master/Alternating_Least_Squares_(ALS)_%26_Bayesian_Personalized_Ranking_(BPR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
#!pip install --upgrade implicit

In [98]:
#!pip install -U "ray[default]"

In [99]:
#!pip install tune-sklearn ray[tune]

In [241]:
import zipfile
import time
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import scipy.sparse as sparse

from datetime import datetime, timedelta


# Loading DataFrames



In [242]:
df_zip = zipfile.ZipFile('/content/drive/MyDrive/RecSys/Orders_Nov22_Jun23.zip')
df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")

<ipython-input-242-6f4658781373>:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")


In [243]:
len(df)

2220299

In [244]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped
0,1547606,01.11.22,NaN,2022FM11,15515778,NET-S M. CHMIELEWSKI,Export Channel (DE),,9433B9X,INK CARTRIDGE SPS,BLACK 370ML 600 DPI INKJET BULK,1037,Consumables,Ink,Supplies,"-15,33",-1.0
1,1547615,01.11.22,NaN,2022FM11,15509465,DIGITAL RIVER IRELAND LIMITED,Export Channel (DE),DIRL,CB31510,LENOVO KEYBOARD PACK,FOR TAB P11-DE,641,Input Devices,Keyboards & Keypads,Printers & Peripherals,"-104,61",-1.0


# Data Preprocessing

In [245]:
# converting the customerid to string
df["BranchCustomerNbr"] = df["BranchCustomerNbr"].astype(str)
# converting the skuid to string
df["Sku"] = df["Sku"].astype(str)

In [246]:
# lines with zero shipment qty
len(df[df["Qty Shipped"] == 0])

113860

In [247]:
df[df["Qty Shipped"] == 0].head(5)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped
24204,4434497,10.01.23,NaN,2023FM01,44475054,EVERIT GMBH,Business Channel,,CD71519,1830 24G 12P CLASS4 POE-STOCK,.,490,Communications & Networking,Lan Switches Managed Layer 2,Networking,"0,00",0.0
35112,4490586,03.02.23,NaN,2023FM06,44840876,KOERBER SUPPLY CHAIN SOFTWARE GMBH,DC-POS,,V36E390,TC21/TC26 HC WHITE FIVE SLOT,CHARGE ONLY CRADLE FIVE DEVICES,6026,AIDC/PoS Accessories & Supplies,Docking Station,"Other (incl. AIDC/POS, V7)","0,00",0.0
35113,4490586,03.02.23,NaN,2023FM06,44840876,KOERBER SUPPLY CHAIN SOFTWARE GMBH,DC-POS,,V36E412,TC21/26 BELT CLIP HC,DISINFECTANT READY TC21/TC26 HC,5706,AIDC/PoS Barcode Scanners,Barcode Scanner Accessories,"Other (incl. AIDC/POS, V7)","0,00",0.0
40384,4437936,17.02.23,NaN,2023FM02,44423409,NOTEBOOKSBILLIGER.DE AG,Consumer Channel,NBB,CF58182,IPHONE 14 PRO 128GB,SPACE BLACK,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0
40385,4437936,17.02.23,NaN,2023FM02,44423409,NOTEBOOKSBILLIGER.DE AG,Consumer Channel,NBB,CF58183,IPHONE 14 PRO 128GB,SILVER,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0


In [248]:
# dropping retours (orders with negative Qty shipped) and zero Qty shipped orders
df = df[df["Qty Shipped"]> 0]

In [249]:
# dropping backlog invoices
df = df[df["Entry Date"] >= '2022-10-01 00:00:00']

In [250]:
len(df)

685862

In [251]:
# implementing a purchase frequency counter helper
df.loc[:, "purchase"] = 1

<ipython-input-251-12c59fb326eb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "purchase"] = 1


## User and sku unqiue list

In [252]:
# Create a numeric user_id and artist_id column
df['BranchCustomerNbr'] = df['BranchCustomerNbr'].astype("category")
df['Sku'] = df['Sku'].astype("category")
df['bcn_id'] = df['BranchCustomerNbr'].cat.codes
df['sku_id'] = df['Sku'].cat.codes

In [253]:
df.head()

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped,purchase,bcn_id,sku_id
7402,4428029,23.11.22,NaN,2022FM12,44504136,THINK ABOUT IT GMBH,Business Channel,,CF58186,IPHONE 14 PRO 256GB,SPACE BLACK,1402,Mobility,Smart Phones,Printers & Peripherals,"1.118,04",1.0,1,4820,33457
7902,4435364,24.11.22,NaN,2022FM11,44423496,KOSATEC COMPUTER GMBH,Mid Market,,406A863,27BN55U 27IN 68.58CM IPS,3840 X 2160 16 : 9 HDMI,320,Display,Lcd Monitor,Printers & Peripherals,"8.432,00",34.0,1,1144,8567
7903,4435457,24.11.22,NaN,2022FM12,44538627,COMPUTACENTER AG & CO. OHG,Business Channel,CAC,CC93753,IPAD 9TH WI-FI + CELL 64GB,10.2IN - A13 CHIP - SPACE GREY,14,Computer Systems,Tablets (Slate Style),System,"1.376,07",3.0,1,6140,23686
8687,4449800,28.11.22,NaN,2022FM12,44880084,PRO-EL GMBH,SMB Channel,,Y340805,SOCKETSCAN S740 UNIVERSAL,BARCODE SCANNER BLACK,5702,AIDC/PoS Barcode Scanners,Handheld Barcode Scanner,"Other (incl. AIDC/POS, V7)","3.895,98",11.0,1,8471,48351
9035,4457156,29.11.22,NaN,2023FM02,44443232,FUJITSU TECHNOLOGY SOLUTIONS GMBH,Business Channel,FTS,I440972,CHERRY MW 4500 BLACK,WIRELESS ERGONOMIC MOUSE USB,601,Input Devices,Mouse,Printers & Peripherals,"15,51",1.0,1,1894,41664


For training 4 different train sets will be implemented for ALS:


*   Summed quantity per SKU
*   Summed purchase frequency based on Entry date count
*   Binary




In [254]:
qty_df = df.groupby(["bcn_id", "sku_id"]).agg({
    "Qty Shipped": "sum"
}).reset_index()

In [255]:
frequency_df = df.groupby(["bcn_id", "sku_id"]).agg({
    "purchase": "sum"
}).reset_index()
frequency_df

,bcn_id,sku_id,purchase
0,0,2276,1
1,0,2277,2
2,0,2796,1
3,0,4417,1
4,0,5535,1
...,...,...,...
284193,9885,1816,2
284194,9885,28315,2
284195,9886,26057,1
284196,9887,28303,1


In [256]:
frequency_m_df = df.groupby(["bcn_id", "sku_id", "FiscalMonth"]).agg({
    "purchase": "mean"
}).reset_index()
frequency_m_df = frequency_m_df.groupby(["bcn_id", "sku_id"]).agg({
    "purchase": "sum"
}).reset_index()
frequency_m_df.purchase.max()

8.0

In [257]:
# add column that represent sku count
def sku_count(df_pl):

  df_pl['sku_count'] = df_pl.groupby('bcn_id')['sku_id'].transform('nunique')

  return df_pl

In [258]:
qty_df = sku_count(qty_df)
frequency_df = sku_count(frequency_df)
frequency_m_df = sku_count(frequency_m_df)

In [259]:
frequency_df

,bcn_id,sku_id,purchase,sku_count
0,0,2276,1,52
1,0,2277,2,52
2,0,2796,1,52
3,0,4417,1,52
4,0,5535,1,52
...,...,...,...,...
284193,9885,1816,2,2
284194,9885,28315,2,2
284195,9886,26057,1,1
284196,9887,28303,1,1


In [260]:
frequency_m_df

,bcn_id,sku_id,purchase,sku_count
0,0,2276,1.0,52
1,0,2277,2.0,52
2,0,2796,1.0,52
3,0,4417,1.0,52
4,0,5535,1.0,52
...,...,...,...,...
284193,9885,1816,1.0,2
284194,9885,28315,1.0,2
284195,9886,26057,1.0,1
284196,9887,28303,1.0,1


In [261]:
# drop customers with less than 2 product as loocv will not work
def sku_drop(df_pl):
  df_pl = df_pl[df_pl.sku_count > 2]
  return df_pl

In [262]:
qty_df = sku_drop(qty_df)
frequency_df = sku_drop(frequency_df)
frequency_m_df = sku_drop(frequency_m_df)

## LOOCV

with K=1 so 1 item per user gets dropped out

In [263]:
def train_test_split(df, holdout_num):
    """ perform training testing split

    @param df: dataframe
    @param holdhout_num: number of items to be held out per user as testing items

    @return df_train: training data
    @return df_test testing data

    """
    # perform deep copy to avoid modification on the original dataframe
    df_train = df.copy(deep=True)
    df_test = df.copy(deep=True)

    # get test set
    df_test = df_test.groupby(['bcn_id']).head(holdout_num).reset_index()

    # get train set
    df_train = df_train.merge(
        df_test[['bcn_id', 'sku_id']].assign(remove=1),
        how='left'
    ).query('remove != 1').drop('remove', 1).reset_index(drop=True)

    # drop index
    df_test = df_test.drop(columns=["index"])

    # sanity check to make sure we're not duplicating/losing data
    assert len(df) == len(df_train) + len(df_test)

    return df_train, df_test

In [264]:
df_qty_train, df_qty_test = train_test_split(qty_df[["bcn_id", "sku_id", "Qty Shipped"]], 1)

<ipython-input-263-adb5a8f5b6e1>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ).query('remove != 1').drop('remove', 1).reset_index(drop=True)


In [265]:
df_frequency_train, df_frequency_test = train_test_split(frequency_df[["bcn_id", "sku_id", "purchase"]], 1)

<ipython-input-263-adb5a8f5b6e1>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ).query('remove != 1').drop('remove', 1).reset_index(drop=True)


In [266]:
df_frequency_m_train, df_frequency_m_test = train_test_split(frequency_m_df[["bcn_id", "sku_id", "purchase"]], 1)

<ipython-input-263-adb5a8f5b6e1>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ).query('remove != 1').drop('remove', 1).reset_index(drop=True)


### Stratifying for same bcns in train test

In [267]:
# check whether there are users in test but not in train
print(len(df_qty_train.bcn_id.unique()), len(df_qty_test.bcn_id.unique()))
print(len(df_frequency_train.bcn_id.unique()), len(df_frequency_test.bcn_id.unique()))
print(len(df_frequency_m_train.bcn_id.unique()), len(df_frequency_m_test.bcn_id.unique()))

5941 5941
5941 5941
5941 5941


In [268]:
# check whether there are skus in test but not in train
print(len(df_qty_train.sku_id.unique()), len(df_qty_test.sku_id.unique()))
print(len(df_frequency_train.sku_id.unique()), len(df_frequency_test.sku_id.unique()))
print(len(df_frequency_m_train.sku_id.unique()), len(df_frequency_m_test.sku_id.unique()))

48021 3338
48021 3338
48021 3338


In [269]:
train_sku_ids = set(df_qty_train.sku_id.unique())
test_sku_ids = set(df_qty_test.sku_id.unique())
test_sku_ids.issubset(train_sku_ids)

False

In [ ]:
test_skus_ids = pd.DataFrame(test_sku_ids,columns=["sku_id"])
test_skus_ids["in_train"] = test_skus_ids.isin(train_sku_ids)
test_skus_ids

In [272]:
def stratify_bcn(train_df, test_df):
  """ remove users from test set, that are not in train
    @params: train and test dfs

    @return stratified_df_test testing data
  """
  # Get the unique 'bcn_id' values from df_train
  trained_bcn_ids = train_df['bcn_id'].unique()
  # Filter out the rows in df_test that have 'bcn_id' values not present in trained_bcn_ids
  stratified_df_test = test_df[test_df['bcn_id'].isin(trained_bcn_ids)]

  return stratified_df_test

In [273]:
def stratify_sku(train_df, test_df):
  """ remove skus from test set that are not in train set
    @params: train and test dfs

    @return stratified_df_test testing data
  """
  # Get the unique 'bcn_id' values from df_train
  train_bcn_ids = train_df['sku_id'].unique()
  # Filter out the rows in df_test that have 'bcn_id' values not present in trained_bcn_ids
  sku_stratified_df_test = test_df[test_df['sku_id'].isin(train_bcn_ids)]

  return sku_stratified_df_test

In [274]:
stratified_df_qty_test = stratify_bcn(df_qty_train, df_qty_test)
sku_stratified_df_qty_test = stratify_sku(df_qty_train, stratified_df_qty_test)

In [275]:
stratified_df_frequency_test = stratify_bcn(df_frequency_train, df_frequency_test)
sku_stratified_df_frequency_test = stratify_sku(df_frequency_train, stratified_df_frequency_test)

In [276]:
stratified_df_frequency_m_test = stratify_bcn(df_frequency_m_train, df_frequency_m_test)
sku_stratified_df_frequency_m_test = stratify_sku(df_frequency_m_train, stratified_df_frequency_m_test)

In [277]:
train_sku_ids_2 = set(df_frequency_m_train.sku_id.unique())
test_sku_ids = set(sku_stratified_df_frequency_m_test.sku_id.unique())
test_sku_ids.issubset(train_sku_ids_2)

True

In [278]:
df_qty_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272872 entries, 0 to 272871
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   bcn_id       272872 non-null  int64  
 1   sku_id       272872 non-null  int64  
 2   Qty Shipped  272872 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 6.2 MB


## Create sparse user x item and item x user matrices

In [279]:
def create_qty_user_item_csr_matrix(df_pl_train, df_pl_test):
  """ create a sparse data matrix with scipy sparse matrix
      @return sparse user x item matrix
  """
  sparse_user_item_train = sparse.csr_matrix((df_pl_train['Qty Shipped'], (df_pl_train['bcn_id'], df_pl_train['sku_id'])))
  sparse_user_item_test = sparse.csr_matrix((df_pl_test['Qty Shipped'], (df_pl_test['bcn_id'], df_pl_test['sku_id'])))

  return sparse_user_item_train, sparse_user_item_test

In [280]:
def create_user_item_csr_matrix(df_pl_train, df_pl_test):
  """ create a sparse data matrix with scipy sparse matrix
      @return sparse user x item matrix
  """
  sparse_user_item_train = sparse.csr_matrix((df_pl_train['purchase'].astype(float), (df_pl_train['bcn_id'], df_pl_train['sku_id'])))
  sparse_user_item_test = sparse.csr_matrix((df_pl_test['purchase'].astype(float), (df_pl_test['bcn_id'], df_pl_test['sku_id'])))

  return sparse_user_item_train, sparse_user_item_test

In [281]:
def create_item_user_csr_matrix(df_pl):
  """ create a sparse data matrix with scipy sparse matrix
      @return sparse user x item matrix
  """
  sparse_item_user = sparse.csr_matrix((df_pl['purchase'].astype(float), (df_pl['sku_id'], df_pl['bcn_id'])))

  return sparse_item_user

In [282]:
sparse_qty_user_item_train, sparse_qty_user_item_test = create_qty_user_item_csr_matrix(df_qty_train, sku_stratified_df_qty_test)
sparse_frequency_user_item_train, sparse_frequency_user_item_test = create_user_item_csr_matrix(df_frequency_train, sku_stratified_df_frequency_test)
sparse_frequency_m_user_item_train, sparse_frequency_m_user_item_test = create_user_item_csr_matrix(df_frequency_m_train, sku_stratified_df_frequency_m_test)

In [283]:
sparse_qty_user_item_train, sparse_qty_user_item_test

(<9881x49112 sparse matrix of type '<class 'numpy.float64'>'
 	with 272872 stored elements in Compressed Sparse Row format>,
 <9881x49035 sparse matrix of type '<class 'numpy.float64'>'
 	with 5327 stored elements in Compressed Sparse Row format>)

##Create unique sku and user lists

In [284]:
skus = df[["sku_id", "Sku" , "Product Descr1", "Product Descr2", "ProductGroupDescription"]].drop_duplicates()

In [285]:
users = df[["bcn_id", "BranchCustomerNbr"]].drop_duplicates()

In [303]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811666 sha256=e2b05374f794d95f255dd41d1eba0b00d48d5e74b158b51b7a8902ff25284140
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [304]:
from surprise import Dataset
from surprise import NormalPredictor
from surprise.model_selection import train_test_split
from surprise import accuracy

In [305]:
qty_df

,bcn_id,sku_id,Qty Shipped,sku_count
0,0,2276,1.0,52
1,0,2277,4.0,52
2,0,2796,4.0,52
3,0,4417,2.0,52
4,0,5535,2.0,52
...,...,...,...,...
284184,9880,7327,10.0,7
284185,9880,8551,5.0,7
284186,9880,26029,1.0,7
284187,9880,30295,1.0,7


In [309]:
data = qty_df[["bcn_id","sku_id","Qty Shipped"]]

In [310]:
# Convert implicit data to Surprise Dataset
# In this case, we use random values for ratings, as the random predictor does not use actual ratings.
# You can set any arbitrary rating values.

# Split data into train and test sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Create the random predictor (Random recommender)
model = NormalPredictor()

# Train the random predictor
model.fit(trainset)

# Generate predictions for testset
predictions = model.test(testset)

# Evaluate Precision at K (P@K) for the random predictor
from surprise import accuracy
from surprise.prediction_algorithms.predictions import Prediction

K = 10  # Top-K items to consider
total_precision = 0.0

for user_id in set([pred.uid for pred in predictions]):
    user_predictions = [pred for pred in predictions if pred.uid == user_id]
    user_predictions.sort(key=lambda x: x.est, reverse=True)  # Sort by predicted rating (est) in descending order
    top_K_predictions = user_predictions[:K]  # Top-K predicted ratings
    relevant_items = [pred.iid for pred in top_K_predictions if pred.r_ui > 0.0]  # Only consider items with positive rating (implicit data)
    precision = len(relevant_items) / K
    total_precision += precision

average_precision = total_precision / len(set([pred.uid for pred in predictions]))

print("Average Precision at 10 (P@10) for the random recommender:", average_precision)

AttributeError: ignored

# **ALS Model**

In [312]:
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import leave_k_out_split, precision_at_k, mean_average_precision_at_k, ndcg_at_k, AUC_at_k, train_test_split
from sklearn.model_selection import GridSearchCV

In [321]:
qty_df_2 = qty_df[["bcn_id","sku_id","Qty Shipped"]]
train_sku_ids = set(df_qty_train.sku_id.unique())
test_sku_ids = set(df_qty_test.sku_id.unique())
test_sku_ids.issubset(train_sku_ids)
sparse_qty_df = sparse.csr_matrix((qty_df_2['Qty Shipped'].astype(float), (qty_df_2['bcn_id'], qty_df_2['sku_id'])))

In [322]:
train, test = train_test_split(sparse_qty_df, train_percentage=0.8)

## ALS Model param tuning

In [287]:
#test if model works on data
%%time
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=10)
model.fit(sparse_qty_user_item_train)

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 9.75 s, sys: 14.1 s, total: 23.9 s
Wall time: 19.5 s


ALS hyperparameters are:


*   factors: The number of latent factors to compute
*   regularization: The regularization factor to use
*   alpha: The weight to give to positive examples
*   iterations: The number of ALS iterations to use when fitting data





In [ ]:
# def validate(sparse_user_item_train, sparse_user_item_test, factors=200, alpha=40, iterations=20, regularization=0.01, show_progress=True):
#     """ Train an ALS model with <<factors>> (embeddings dimension)
#     for <<iterations>> over matrices and validate with MAP@10
#     """

#     model = implicit.als.AlternatingLeastSquares(factors=factors,
#                                                  alpha=alpha,
#                                                  iterations=iterations,
#                                                  regularization=regularization)

#     model.fit(sparse_user_item_train, show_progress=show_progress)

#     # The MAPK by implicit doesn't allow to calculate allowing repeated items, which is the case.
#     # TODO: change MAP@10 to a library that allows repeated items in prediction
#     map10 = mean_average_precision_at_k(model, sparse_user_item_train, sparse_user_item_test, K=10, show_progress=show_progress, num_threads=4)
#     print(f"Factors: {factors:>3} - Confidence: {alpha:>9} - Regularization: {regularization:4.3f} - Iterations: {iterations:>2} ==> MAP@10: {map10:6.5f}")
#     return map10

In [ ]:
# %%time
# best_map10 = 0
# for factors in [10, 50, 100, 150, 200]:
#   for alpha in [10, 20, 30, 40]:
#     for iterations in [10, 15, 20]:
#       for regularization in [0.01, 0.05, 0.1]:
#           map10 = validate(sparse_user_item_train, sparse_user_item_test, factors, alpha, iterations, regularization, show_progress=False)
#           if map10 > best_map10:
#             best_map10 = map10
#             best_params = {'factors': factors, 'confidence': alpha, 'iterations': iterations, 'regularization': regularization}
#             print(f"Best MAP@10 found. Updating: {best_params}")

In [ ]:
# from sklearn.base import BaseEstimator
# from sklearn.model_selection import GridSearchCV

# class WrappedALS(BaseEstimator):
#     def __init__(self, factors=100, regularization=0.01, iterations=20, alpha=40):
#         self.factors = factors
#         self.regularization = regularization
#         self.iterations = iterations
#         self.alpha = alpha
#         self.als_model = AlternatingLeastSquares(
#             factors=self.factors,
#             regularization=self.regularization,
#             iterations=self.iterations,
#             calculate_training_loss=True,
#         )

#     def fit(self, X, y=None):
#         self.als_model.fit(X)
#         return self

#     def predict(self, X):
#         return self.als_model.user_factors.dot(self.als_model.item_factors.T)

#     def score(self, X, y=None):
#         precision = precision_at_k(self.als_model, X, X.T, K=10)
#         return precision_at_k

# # Define the hyperparameter ranges
# param_grid = {
#     'factors': [50, 100, 150, 200],
#     'alpha': [20, 40],
#     'iterations': [10, 25, 20],
#     'regularization': [0.01, 0.05]
# }

# # Initialize WrappedALS model
# wrapped_als_model = WrappedALS()

# # Initialize GridSearchCV
# grid_search = GridSearchCV(wrapped_als_model, param_grid, cv=5, verbose=1)

# # Fit the GridSearchCV object to the data
# grid_search.fit(sparse_user_item_train)

# # Get the best ALS model and its hyperparameters
# best_als_model = grid_search.best_estimator_
# best_params = grid_search.best_params_

# print("Best Hyperparameters:")
# print(best_params)

In [ ]:
from ray.tune.sklearn import TuneSearchCV, TuneGridSearchCV

# Create the Implicit ALS model
model = implicit.als.AlternatingLeastSquares()

param_grid = {
    'factors': [50, 100, 200],  # number of latent factors
    'iterations': [10, 20, 30],  # number of iterations
    'regularization': [0.01, 0.1, 1.0],  # regularization strength
    'alpha': [15, 25, 35]  # confidence scaling factor
}

best_precision = 0.0
best_params = {}

# Perform grid search using the built-in grid search
for factors in param_grid['factors']:
    for iterations in param_grid['iterations']:
        for regularization in param_grid['regularization']:
            for alpha in param_grid['alpha']:
                # Set the hyperparameters for the model
                model.params['factors'] = factors
                model.params['iterations'] = iterations
                model.params['regularization'] = regularization
                model.params['alpha'] = alpha

                # Train the model
                model.fit(train)

                # Evaluate the model on the test set using precision_at_k
                precision = implicit.evaluation.precision_at_k(model, train.T.tocsr(), test.T.tocsr(), K=10)

                # Check if current hyperparameters give better precision
                if precision.mean() > best_precision:
                    best_precision = precision.mean()
                    best_params = {'factors': factors, 'iterations': iterations, 'regularization': regularization, 'alpha': alpha}

print("Best Hyperparameters:", best_params)
print("Best Precision:", best_precision)

AttributeError: ignored

In [328]:
%%time
model_1 = implicit.als.AlternatingLeastSquares(factors=200,
                                                 alpha=25,
                                                 iterations=20,
                                                 regularization=0.01)
model_1.fit(sparse_qty_user_item_train)
prec10 = precision_at_k(model_1, sparse_qty_user_item_train, sparse_qty_user_item_test, K=10)
print(prec10)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5327 [00:00<?, ?it/s]

0.10061948563919655
CPU times: user 1min 41s, sys: 1min 43s, total: 3min 24s
Wall time: 2min 17s


In [330]:
map10 = mean_average_precision_at_k(model_1, sparse_qty_user_item_train, sparse_qty_user_item_test, K=10)
map10

  0%|          | 0/5327 [00:00<?, ?it/s]

0.045713064025434895

In [337]:
ndcg10 = ndcg_at_k(model_1, sparse_qty_user_item_train, sparse_qty_user_item_test, K=10)
ndcg10

  0%|          | 0/5327 [00:00<?, ?it/s]

0.05855987070899835

In [332]:
%%time
model_2 = implicit.als.AlternatingLeastSquares(factors=200,
                                                 alpha=25,
                                                 iterations=20,
                                                 regularization=0.01)
model_2.fit(sparse_frequency_user_item_train)
prec10 = precision_at_k(model_2, sparse_frequency_user_item_train, sparse_frequency_user_item_test, K=10)
print(prec10)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5327 [00:00<?, ?it/s]

0.11807771728928101
CPU times: user 1min 33s, sys: 1min 33s, total: 3min 7s
Wall time: 2min 6s


In [334]:
map10 = mean_average_precision_at_k(model_2, sparse_qty_user_item_train, sparse_qty_user_item_test, K=10)
map10

  0%|          | 0/5327 [00:00<?, ?it/s]

0.05488638293688037

In [338]:
ndcg10 = ndcg_at_k(model_2, sparse_qty_user_item_train, sparse_qty_user_item_test, K=10)
ndcg10

  0%|          | 0/5327 [00:00<?, ?it/s]

0.06811422221085194

In [335]:
%%time
model_3 = implicit.als.AlternatingLeastSquares(factors=200,
                                                 alpha=25,
                                                 iterations=20,
                                                 regularization=0.01)
model_3.fit(sparse_frequency_m_user_item_train)
prec10 = precision_at_k(model_3, sparse_frequency_m_user_item_train, sparse_frequency_m_user_item_test, K=10)
print(prec10)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5327 [00:00<?, ?it/s]

0.11939177773606158
CPU times: user 1min 36s, sys: 1min 37s, total: 3min 13s
Wall time: 2min 14s


In [341]:
map10 = mean_average_precision_at_k(model_3, sparse_frequency_m_user_item_train, sparse_frequency_m_user_item_test, K=10)
map10

  0%|          | 0/5327 [00:00<?, ?it/s]

0.055761305836394895

In [339]:
ndcg10 = ndcg_at_k(model_3, sparse_frequency_m_user_item_train, sparse_frequency_m_user_item_test, K=10)
ndcg10

  0%|          | 0/5327 [00:00<?, ?it/s]

0.07064848797354266

In [301]:
auc10 = AUC_at_k(model_1, sparse_qty_user_item_train, sparse_qty_user_item_test, K=10)
auc10

  0%|          | 0/5327 [00:00<?, ?it/s]

0.5510566299723826

In [298]:
%%time
model_2 = implicit.als.AlternatingLeastSquares(factors=200,
                                                 alpha=25,
                                                 iterations=20,
                                                 regularization=0.1)
model_2.fit(sparse_qty_user_item_train)
prec10 = precision_at_k(model_2, sparse_qty_user_item_train, sparse_qty_user_item_test, K=10)
print(prec10)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5327 [00:00<?, ?it/s]

0.1000563168762906
CPU times: user 1min 39s, sys: 1min 40s, total: 3min 20s
Wall time: 2min 11s


In [324]:
%%time
model_3 = implicit.als.AlternatingLeastSquares(factors=200,
                                                 alpha=25,
                                                 iterations=20,
                                                 regularization=0.1)
model_3.fit(train)
prec10 = precision_at_k(model_3, train, test, K=10)
print(prec10)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/4703 [00:00<?, ?it/s]

0.2519453207150368
CPU times: user 1min 30s, sys: 1min 28s, total: 2min 59s
Wall time: 1min 58s


## ALS Best Model Performance

In [ ]:
precision_als = precision_at_k(model_3, sparse_user_item_train, sparse_user_item_test, K=10, num_threads=10)
precision_als

  0%|          | 0/6611 [00:00<?, ?it/s]

0.12554832854333686

In [ ]:
ndcg = ndcg_at_k(model_3, sparse_user_item_train, sparse_user_item_test, show_progress=True, K=10, num_threads=10)
ndcg

  0%|          | 0/6611 [00:00<?, ?it/s]

0.07566750354043612

## ALS Model Recommendations

In [ ]:
 # Assuming you want recommendations for user with ID 10
user_id = 275

# Now you can call the recommend function
recommended = model_3.recommend(user_id, user_items[user_id], N=13, filter_already_liked_items=True)
rec_tab = pd.DataFrame(recommended)
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"conf"})
rec_tab = rec_tab.merge(skus, on="sku_id", how="left")
rec_tab.sort_values(by="conf", ascending=False)

,sku_id,conf,Sku,Product Descr1,Product Descr2,ProductGroupDescription
0,22954.0,0.766616,CC47067,TRELLIX PROTECT STANDARD,COM SUB 1Y SUB BS A:5-250,Other Security Software
1,13633.0,0.620980,677QL89,CATALYST 9200L 24-PORT POE+,4 X 10G NETWORK ESSENTIALS,Lan Switches Managed Layer 2
2,22965.0,0.567549,CC47441,SEC FOR MS EXCH EPO 1:1BZ,COM SUB 1Y SUB BS A:5-250,Other Security Software
3,13652.0,0.523446,677QP85,600W AC CONFIG 5,POWER SUPPLY,Power Supplies
4,13452.0,0.445097,677NW74,AIRONET DNA ESSENTIALS,3 YEAR TERM LICENSE,Network Management & Utility Software
5,13953.0,0.427214,677SN67,CATALYST 802.11AX AP EXT. ANTEN,4X4:4 MIMO IOT BT5MGIG RHL-E DOM,Wireless Lan Access Points (Ap)
6,13804.0,0.417205,677RP77,CATALYST 802.11AX APINT ANTENNA,4X4:4 MIMO IOT BT5MGIG USB E DOM,Wireless Lan Access Points (Ap)
7,23154.0,0.409915,CC71596,SEC FOR MS EXCH EPO 1:1BZ,GOV SUB 1Y SUB BS B:251-1000,Other Security Software
8,13769.0,0.407640,677RG09,CISCO CATALYST 802.11AX AP INT,ANTENNA 4X4:4 MIMO BT 5 MGIG,Wireless Lan Access Points (Ap)
9,13471.0,0.404087,677P577,1000BASE-LX/LH SFP TRANSCEIVER,MODULE MMF/SMF 1310NM DOM,"Transceivers (Gbic, Spf) & Media Converters"


In [ ]:
sku_stratified_df_test[200:400]

,bcn_id,sku_id,purchase
232,269,1450,1
234,271,17995,1
235,273,10156,1
236,274,20522,1
237,275,13643,1
...,...,...,...
471,606,7753,1
472,608,9709,1
473,609,24502,1
474,610,690,1


In [ ]:
sku_stratified_df_test[sku_stratified_df_test.bcn_id == user_id]

,bcn_id,sku_id,purchase
237,275,13643,1


In [ ]:
user_purchases = pd.DataFrame(df[df.bcn_id == user_id][["Product Descr1","Sku", "ProductGroupDescription", "purchase"]].groupby(
                              ["Sku","Product Descr1", "ProductGroupDescription"])["purchase"].sum().sort_values(ascending=False))
user_purchases[0:20]

,,,purchase
Sku,Product Descr1,ProductGroupDescription,
CB32850,DNA ESSENTIALS 3 YEAR LICENSE,Network Management & Utility Software,3
677SB70,C9300L CISCO DNA ESSENTIALS,Network Management & Utility Software,2
CC70881,TRELLIX STANDARD 1:1BZ,Other Security Software,2
CC71595,SEC FOR MS EXCH EPO 1:1BZ,Other Security Software,2
CE92527,SOLN SUPP SW SUB C9300 DNA,Services And Warranties,1
677QP75,C9200L DNA ESSENTIALS 24-PORT,Network Management & Utility Software,1
CC72388,COMP DATA PROT ADV P:1BZP+,Other Security Software,1
677QZ00,715W AC 80+ PLATINUM CONFIG 1,Power Supplies,1
CB32957,CATALYST 9300X 24X25G FIBER,Lan Switches Managed Layer 3,1


In [ ]:
#Get similar items
item_id = 40
n_similar = 10
similar = model_2.similar_items(item_id, n_similar)
print(similar)

(array([36081, 33909, 28762, 27736, 26413,  7125,   790, 27609, 26473,
        7808], dtype=int32), array([1.        , 1.        , 1.        , 1.        , 0.99999994,
       0.99999994, 0.99999994, 0.9999999 , 0.9999999 , 0.9999999 ],
      dtype=float32))


# BPR Model

### Negative sampling


In [ ]:
def negative_sampling(bcn_ids, sku_ids, items, n_neg):
    """This function creates n_neg negative labels for every positive label

    @param user_ids: list of user ids
    @param sku_ids: list of sku ids
    @param items: unique list of sku ids
    @param n_neg: number of negative labels to sample

    @return df_neg: negative sample dataframe

    """

    neg = []
    ui_pairs = zip(bcn_ids, sku_ids)
    records = set(ui_pairs)

    # for every positive label case
    for (u, i) in records:
        # generate n_neg negative labels
        for _ in range(n_neg):
            j = np.random.choice(items)
            # resample if the movie already exists for that user
            while (u, j) in records:
                j = np.random.choice(items)
            neg.append([u, j, 0])

    # convert to pandas dataframe for concatenation later
    df_neg = pd.DataFrame(neg, columns=['bcn_id', 'sku_id', 'purchase'])

    return df_neg

In [ ]:
neg_train = negative_sampling(
    bcn_ids=df_train.bcn_id.unique(),
    sku_ids=df_train.sku_id.unique(),
    items=grouped_df_binary.sku_id.unique(),
    n_neg=5
)

In [ ]:
# create final training and testing sets
#df_train = df_train[['bcn_id', 'sku_id']].assign(purchase=1)
df_train = pd.concat([df_train, neg_train], ignore_index=True)

In [ ]:
df_train_binary = pd.concat([df_train_binary, neg_train], ignore_index=True)

In [ ]:
sparse_user_item_train_binary = create_user_item_csr_matrix(df_train_binary)
sparse_user_item_test_binary = create_user_item_csr_matrix(sku_stratified_df_test_binary)

In [ ]:
stratified_df_test_binary = stratify_bcn(df_train_binary, df_test_binary)
sku_stratified_df_test_binary = stratify_sku(df_train_binary, stratified_df_test_binary)

In [ ]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808298 sha256=81cfe330993df21442639f92ff6ef44e554a663825b1c7dceee9ed5a502d45cb
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
binary_df = qty_df[["bcn_id", "sku_id"]]
binary_df["purchase"] = 1
binary_df

,bcn_id,sku_id,purchase
0,0,2276,1
1,0,2277,1
2,0,2796,1
3,0,4417,1
4,0,5535,1
...,...,...,...
284184,9880,7327,1
284185,9880,8551,1
284186,9880,26029,1
284187,9880,30295,1


In [ ]:
from implicit.bpr import BayesianPersonalizedRanking as BPR
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.model_selection import train_test_split
from lightfm.datasets import fetch_movielens

## Implicit BPR

In [ ]:
model_bpr = BPR(factors=200, learning_rate=0.01, regularization=0.01,
                verify_negative_samples=True)

In [ ]:
model_bpr.fit(sparse_user_item_train_binary)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
auc = AUC_at_k(model_bpr, sparse_user_item_train_binary, sparse_user_item_test_binary, K=10, show_progress=True)
auc

  0%|          | 0/6611 [00:00<?, ?it/s]

0.5070077130105727

In [ ]:
map10 = mean_average_precision_at_k(model_bpr, sparse_user_item_train_binary, sparse_user_item_test_binary, K=10, show_progress=True,
                                    num_threads=10)
map10

  0%|          | 0/6611 [00:00<?, ?it/s]

0.0043608175887709984

In [ ]:
 # Assuming you want recommendations for user with ID 10
user_id = 275

# Now you can call the recommend function
recommended = model_bpr.recommend(user_id, user_items[user_id], N=10, filter_already_liked_items=True)
rec_tab = pd.DataFrame(recommended)
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"conf"})
rec_tab = rec_tab.merge(skus, on="sku_id", how="left")
rec_tab.sort_values(by="conf", ascending=False)

,sku_id,conf,Sku,Product Descr1,Product Descr2,ProductGroupDescription
0,10965.0,0.856612,54153C6,WORRY FREE 5 SERVICES ADV,RNW LIZ 12 M - 0011 - 0025 USER,Other Security Software
1,43129.0,0.827776,N170825,LANCOM GS-3126XP,MANAGED LAYER-3-SWITCH 24 PORTS,Lan Switches Managed Layer 3
2,43101.0,0.818499,N170742,LANCOM ADVANCED VPN CLIENT,WIN,Network Management & Utility Software
3,37568.0,0.798571,CG25158,BU+REPL ENT COM,PR MIG FR BRE SB,Back-Up Software
4,11065.0,0.794843,5417HQ3,WORRY FREE 9 STANDARD,ADD LIZ 12 M - 0026 - 0050 USER,"Virus Detection, Antivirus Software"
5,24357.0,0.791499,CD12272,POWEREDGE R350 SERVER,CTO CONFIG,Servers
6,29334.0,0.791086,CE57605,FRACHTKOSTEN / FREIGHT,,Vendor Specific Configuation Fees
7,14868.0,0.791080,803TS37,OFFICE STD,OLV SA NL 1YACQY1 ADP,Services And Warranties
8,43986.0,0.790560,T130SC0,CYBER BACKUP STD SERVER,1Y RNW PCS ESD,Disaster/File Recovery Software
9,28554.0,0.770177,CE40398,AUTOCAD LT 2023 NEW,SGL-US 3YR SUBSCRIPTION,Cad/Cam/Cae Software


In [ ]:
sku_stratified_df_test[sku_stratified_df_test.bcn_id == user_id]

,bcn_id,sku_id,purchase
237,275,13643,1


## LightFM BPR

In [ ]:
dataset = Dataset()
dataset.fit(user_ids, sku_ids)

In [ ]:
movielens = fetch_movielens()
movielens

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 90570 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 9430 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1'),
 'item_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1')}

In [ ]:
train = movielens["train"]
pd.DataFrame(train.toarray())

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,5,3,4,3,3,5,4,1,5,3,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
939,0,0,0,2,0,0,4,5,3,0,...,0,0,0,0,0,0,0,0,0,0
940,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
941,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
item_feature_lables = movielens["item_feature_labels"]
item_feature_lables

array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1')

In [ ]:
interactions, weights = dataset.build_interactions(zip(user_ids, sku_ids))

In [ ]:
# Split the interactions matrix into train and test sets
train_interactions, test_interactions = train_test_split(interactions, test_size=0.2, random_state=42)

In [ ]:
model = LightFM(learning_rate=0.05, loss='bpr')

In [ ]:
model.fit(train_interactions, epochs=10)

In [ ]:
sku_stratified_df_test[sku_stratified_df_test.bcn_id == user_id]

## BPR Model param tuning